In [1]:
# btmのrootingの練習をする

%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
import numpy as np

from PerplexityMoE import PerplexityMoE

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
moe=PerplexityMoE()


model_name_list =[ 
    "../models/hf/0316mix_part1_eng",
    "../models/hf/0316mix_part2_mc4",
    "../models/hf/0316mix_fin",
                  ]

for model_name in model_name_list:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name, device_map="auto",
        torch_dtype=torch.float16
    )

    moe.append_ELM(model,tokenizer)

In [22]:
text="こんにちは､元気ですか"


tokenized_input = tokenizer.encode(
    text, add_special_tokens=False, return_tensors="pt"
).to(model.device)
with torch.inference_mode():
    output = model(tokenized_input, labels=tokenized_input[:])
ppl = torch.exp(output.loss)
ppl.item()
#tokenized_input

295.25

In [19]:
output.logits.shape

torch.Size([1, 5, 65024])

In [47]:
#moe.set_coefs(np.array([1,0.5,1]))
moe.set_coefs([1,1,1])

text_list=[
    "hello, I'm John",
    "今日は晴れてるから",
    "ブログを書きました",
    "global warming ",
    "地球温暖化を防ぐに",
    "ガリレオによるピサの斜塔実験とは",
    "英語: He is a good man. 日本語: ",
      ]
response_list=[]
ppl_list=[]
model_list=[]
for text in text_list:
    print("\n\n-----")
    print("input: ", text)
    response,ppl,model_id=moe.ask(text)
    print("response: ",response)
    response_list.append(response)
    ppl_list.append(ppl)
    model_list.append(model_id)

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




-----
input:  hello, I'm John
perplexity list
0 302.25
1 222.75
2 161.75
model id 2 is used


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


response:  hello, I'm John.

In 1986 the film was remade as The Last of Us (with a new title) and starred Michael Caine in his first leading role on Broadway at New York City’s Globe Theatre Festival where it ran for two seasons before being transferred to Los Angeles after its run by David Letterman from San Francisco Opera Company with Robert Redford Jr. It received an Oscar nomination that same year but lost outright when director Mark Levinson left him due $25


-----
input:  今日は晴れてるから
perplexity list
0 411.5
1 278.25
2 358.75
model id 1 is used


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

response:  global warming 
The climate of the region is very cool and humid. The temperature in this area can be quite cold, but it's not as hot or dry because there are many wind blown from a nearby mountain range to keep up with rain until late summer when temperatures rise above freezing point during winter months (which makes for an extremely pleasant atmosphere). This means that we have been able to see all weather conditions on our own since at least one year ago; however these factors make us feel comfortable


-----
input:  地球温暖化を防ぐに
perplexity list
0 10648.0
1 4372.0
2 3174.0
model id 2 is used


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


response:  地球温暖化を防ぐに、その対策は?
2018年5月3日(木)の「エコノミスト」の記事で紹介した、「地球温暖化防止のための行動指針」(以下、ガイドラインと略記する。)について紹介します。「地球温暖化を防止するための行動指針」とは、環境省が毎年発行している『環境白書』や、『地球環境問題に関する意識調査』(平成9年度～令和元年度)、そして、経済産業省による『省エネ・省エネルギー基準等検討会報告書』などをまとめたものになります。『環境


-----
input:  ガリレオによるピサの斜塔実験とは
perplexity list
0 1589.0
1 459.0
2 159.875
model id 2 is used


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


response:  ガリレオによるピサの斜塔実験とは、1980年(昭和55)に始まった。

このプロジェクトは、2つの主要な目的があった。ひとつ目は、ガリレオが「ガリレイ・タワー」の建設を計画するきっかけとなった、17世紀のヴェネツィアで起こった地震と津波の研究である。「ガリレオ・タワー」建設の目的は、「地震や洪水などの自然災害から身を守ること」、「地震によって建物内の空間構造が変化しないこと」「地震により建物の倒壊を防ぐために耐震性を高めることなど」であった。もう一つ


-----
input:  英語: He is a good man. 日本語: 
perplexity list
0 115.5625
1 64.3125
2 49.3125
model id 2 is used
response:  英語: He is a good man. 日本語: 1985年04月26日
【翻訳】 「私は、この国で何をしたいのか」と聞かれたら、「それは、私がやりたいことじゃない。ただの遊びだ!」と答えた人は、その通りです。「あなたは私のことを好きですか?」という質問に、「あなたが好きなのはあなただけではありませんよ!」「あなたのことが好きですか?それとも嫌いでしょうか?」といったように答えてみてください。
また、自分の意見を述べても構いませんが、相手の考えていることはよく


In [48]:
import pandas as pd
# DataFrameを初期化（ppl_listを文字列で保存するための列も追加）
# 各行のデータを保存するためのリストを初期化
data = []

# テキストリストをループし、各テキストに対する応答とppl_listを取得
for text,response,ppl in zip(text_list,response_list,ppl_list):
    ppl=[int(i) for i in ppl]
    data.append([text, response, ppl])

# リストからDataFrameを作成
df = pd.DataFrame(data, columns=['Input', 'Response', 'PPL_List'])
df["model id"]=model_list
# DataFrameを表示
df

,Input,Response,PPL_List,model id
0,"hello, I'm John","hello, I'm John.\n\nIn 1986 the film was remad...","[302, 222, 161]",2
1,今日は晴れてるから,今日は晴れてるから、いい天気だ。\n2016年5月3日 (水) | 固定リンク 【楽天市場】...,"[411, 278, 358]",1
2,ブログを書きました,ブログを書きました。\n2016-09-30(日) | 編集 | コメント (4)| トラッ...,"[1127, 620, 793]",1
3,global warming,global warming \nThe climate of the region is ...,"[274, 258, 201]",2
4,地球温暖化を防ぐに,地球温暖化を防ぐに、その対策は?\n2018年5月3日(木)の「エコノミスト」の記事で紹介し...,"[10648, 4372, 3174]",2
5,ガリレオによるピサの斜塔実験とは,ガリレオによるピサの斜塔実験とは、1980年(昭和55)に始まった。\n\nこのプロジェクト...,"[1589, 459, 159]",2
6,英語: He is a good man. 日本語:,英語: He is a good man. 日本語: 1985年04月26日\n【翻訳】 「...,"[115, 64, 49]",2


In [37]:
data

[]

In [39]:
response_list

[]

In [46]:
model_Id

2